In [118]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [119]:
play_df = pd.read_csv('plays.csv')
team_abbreviations = list(play_df['possessionTeam'].unique())

new_team_abbr = [team.lower() for team in team_abbreviations]

print("Available teams - ")
print(new_team_abbr)

Available teams - 
['atl', 'phi', 'pit', 'cle', 'cin', 'ind', 'ten', 'mia', 'bal', 'buf', 'ne', 'hou', 'jax', 'nyg', 'no', 'tb', 'was', 'ari', 'car', 'dal', 'gb', 'chi', 'nyj', 'det', 'oak', 'la', 'min', 'lac', 'kc', 'sf', 'den', 'sea']


In [120]:
def get_drive_info(game_ids):
    
    results = []
    
    for game in game_ids:
        
        my_url = 'https://www.espn.com/nfl/playbyplay?gameId={}'.format(game)

        team_dict = {}

        req = requests.get(my_url)
        my_text = req.text
        soup = bs(my_text, 'html.parser')

        headlines = [item.get_text() for item in soup.find_all('span', class_='headline')]

        sub_soup = soup.find(id='main-container')
        logos = sub_soup.find_all('span', class_='home-logo')

        team = re.compile("nfl/500/([a-z]{3})")

        team_abbr = []

        for stuff in logos:
            team_logo = team.findall(str(stuff))
            
            if team_logo:
                team_logo = team_logo[0]
            else:
                pass

            team_abbr.append(team_logo)

        detail_soup = sub_soup.find_all('span',class_='drive-details')

        yard_pattern = re.compile("(-?[0-9]*) yards?")
        yards = []
        for detail in detail_soup:
            deets = detail.get_text()
            yards.append(yard_pattern.findall(deets)[0])

        for i in range(len(headlines)):
            detail_tuple = (team_abbr[i],headlines[i],yards[i])
            results.append(detail_tuple)

            
    return results
        
        

In [121]:
def get_game_ids(team_abbr):
    game_ids = []
    
    my_url = 'https://www.espn.com/nfl/team/schedule/_/name/{}/season/2018'.format(team_abbr)

    req = requests.get(my_url)
    my_text = req.text
    soup = bs(my_text, 'html.parser')
    
    sub_soup = soup.find('section',class_='pt0')
    
    links = sub_soup.find_all('a',class_='AnchorLink')
    
    gameID = re.compile("gameId/([0-9]*)")
    
    for stuff in links:
        stuff = str(stuff)
        
        game_id = gameID.findall(stuff)
        
        if game_id:
            game_ids.append(game_id[0])
    
    return game_ids


In [122]:
ids= get_game_ids('bal')

In [123]:
team_list = get_drive_info(ids)

In [124]:
team_list[:20]

[('bal', 'Punt', '21'),
 ('lac', 'Punt', '7'),
 ('bal', 'Fumble', '-14'),
 ('lac', 'Field Goal', '12'),
 ('bal', 'Punt', '13'),
 ('lac', 'Field Goal', '7'),
 ('bal', 'Punt', '8'),
 ('lac', 'Punt', '29'),
 ('bal', 'Interception', '21'),
 ('lac', 'Field Goal', '37'),
 ('bal', 'Punt', '6'),
 ('lac', 'Field Goal', '58'),
 ('lac', 'Missed FG', '3'),
 ('bal', 'Punt', '-10'),
 ('lac', 'Fumble', '9'),
 ('bal', 'Field Goal', '6'),
 ('lac', 'Punt', '4'),
 ('bal', 'Missed FG', '8'),
 ('lac', 'Touchdown', '60'),
 ('bal', 'Punt', '-9')]